In [1]:
%matplotlib widget

%config Completer.use_jedi = False
import os

import time as time
from time import sleep

import matplotlib.pyplot as plt
import numpy as np
import qcodes as qc
import math as math
from qcodes import (
    Station,
    Measurement,
    experiments,
    initialise_database,
    initialise_or_create_database_at,
    load_by_guid,
    load_by_run_spec,
    load_experiment,
    load_last_experiment,
    load_or_create_experiment,
    new_experiment,
)

from qcodes.dataset.plotting import plot_dataset, plot_by_id
from qcodes.logger.logger import start_all_logging
import datetime

from pandas import Series

print ("Done")

Done


In [2]:
### foldering
indexing_folder = r'T:\mc2-qdp-gold\Ivo\_Science\2103 CPS advanced resonators NbTiN\RodriCPS\Devices\Data'
base_resonator = 'L10_w1_g2_highQc'
base_path = os.path.join(indexing_folder, base_resonator)
print(base_path)

T:\mc2-qdp-gold\Ivo\_Science\2103 CPS advanced resonators NbTiN\RodriCPS\Devices\Data\L10_w1_g2_highQc


In [3]:
### functions and important values

central_value = 6.256e9
span_value = 100e6



# Importing, logging and initialising

In [5]:
station = qc.Station() # start a nice station for all instruments
start_all_logging()
print ("Done")

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\Lab\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\Lab\.qcodes\logs\220317-10980-qcodes.log
Done


In [6]:
from qcodes.instrument_drivers.Lakeshore.Model_372 import Model_372

lake = Model_372('lakeshore_372', 'TCPIP::192.168.0.115::7777::SOCKET') #Embla
heater = lake.sample_heater
station.add_component(lake)
station.add_component(heater)

#################
lake.ch01.units('kelvin'); lake.ch01.enabled(False)
lake.ch02.units('kelvin'); lake.ch02.enabled(False)

lake.ch05.units('kelvin'); lake.ch05.enabled(False)
lake.ch06.units('kelvin'); lake.ch06.enabled(True)
heater.output('off')
heater.output_range('off')
heater.mode('off')

Connected to: LSCI 372 (serial:LSA2807, firmware:1.4) in 0.22s


In [8]:
import qcodes_contrib_drivers.drivers.Keysight.Keysight_P5004A as P5004A

VNA = P5004A.P5004A("VNA", 'TCPIP0::DHCP2-100217::hislip_PXI10_CHASSIS1_SLOT1_INDEX0::INSTR')
#station.add_component(VNA)
print ("Done\n")
#VNA.print_readable_snapshot(update=False)
#VNA.parameters

Connected to: Keysight Technologies P5004A (serial:MY58100298, firmware:A.14.30.06) in 0.13s
Startup commencing
Startup finished
Done



In [ ]:
import spi_rack
from spi_rack.spi_rack import spi_rack
from spi_rack import switch

dac_list = []
for i in range(1,17):
    dac_list.append(f'dac{i}')
    
spi = spi_rack('SPI Rack', 'COM5')
spi.add_spi_module(1, "D5a", "DAC", dac_names = dac_list, reset_voltages = True) # Alias =  DAC 
dac = spi.DAC

print("Done!\n")

In [ ]:
from spi_rack.switch import mw_switch_class

switch_dac_list = []
for i in range (1,7):
    switch_dac_list.append(eval(f'dac.dac{i}'))
        # just does this:
        # switch_dac_list = [dac.dac1, dac.dac2, dac.dac3, dac.dac4, dac.dac5, dac.dac6]

mw_switch = mw_switch_class(name = "switch",
                           dac_list = switch_dac_list,
                           turn_ramp_off = True,
                           reset_switch = False
                           )

#mw_switch.open_port(3)
print("Done!\n")

# Initial temperature sweep while cooling down

In [9]:
high_T_path =  "\Temperature dependence\High_T"
filename = "highT"
data_pth = base_path + high_T_path

T_MXC   = (lake.ch06.temperature())
VNA.receiver_gain('Low')
print("Temperature is {}K".format(T_MXC))

while lake.ch06.temperature() > 4:
    if abs(lake.ch06.temperature() - T_MXC) > 0.5:
        T_MXC   = (lake.ch06.temperature()) #log the MXC temperature
        print("Temperature is {}K".format(T_MXC))
        VNA.measure_S21(4e9, 8e9, 10001, -10, 15, 10e3, 'ss')
        VNA.save_data(data_pth, filename, int(round(1e3*T_MXC)) )
        sleep(30)
    else:
        sleep(30)
        print("Waiting, T is {}K".format(lake.ch06.temperature()))


Temperature is 1.7174K


In [9]:
for ch in lake.channels:
    if ch.enabled():
        print(f'Temperature of {ch.short_name} ({"on" if ch.enabled() else "off"}): {ch.temperature()} {ch.units()}')
        
lake.ch01.units('kelvin'); lake.ch01.enabled(False)
lake.ch02.units('kelvin'); lake.ch02.enabled(False)

lake.ch05.units('kelvin'); lake.ch05.enabled(False)
lake.ch06.units('kelvin'); lake.ch06.enabled(True)


Temperature of ch06 (on): 3.12614 kelvin


In [10]:
T_MXC   = (lake.ch06.temperature())


med_T_path =  "\Temperature dependence\High_T"
filename = "medT"
data_pth = base_path + med_T_path

VNA.receiver_gain('Low')
print("Temperature is {}K".format(T_MXC))

while True:
    while lake.ch06.temperature() > 0.1:
        if abs(lake.ch06.temperature() - T_MXC) > 0.25:
            T_MXC   = (lake.ch06.temperature()) #log the MXC temperature
            print("Temperature is {}K".format(T_MXC))
            VNA.measure_S21(5.5e9, 6.5e9, 10001, -10, 15, 10e3, 'ss')
            VNA.save_data(data_pth, filename, T_MXC*1000)
            sleep(20)
        else:
            sleep(20)
            print("Waiting, T is {}K".format(lake.ch06.temperature()))
    break
    


Temperature is 1.71919K
Waiting, T is 1.71779K
Waiting, T is 1.71786K
Waiting, T is 1.71664K
Waiting, T is 1.71718K
Waiting, T is 1.7184K
Waiting, T is 1.71821K
Waiting, T is 1.7377K
Waiting, T is 1.91734K
Waiting, T is 1.95666K
Waiting, T is 1.93193K
Waiting, T is 1.97507K
Temperature is 1.97534K
Waiting, T is 1.90606K
Waiting, T is 1.83961K
Waiting, T is 1.79126K
Waiting, T is 1.76113K
Waiting, T is 1.7424K
Waiting, T is 1.72126K
Temperature is 1.72126K
Waiting, T is 1.66313K
Waiting, T is 1.64038K
Waiting, T is 1.61931K
Waiting, T is 1.59733K
Waiting, T is 1.56406K
Waiting, T is 1.54246K
Waiting, T is 1.48875K
Waiting, T is 1.46077K
Temperature is 1.46077K
Waiting, T is 1.32685K
Waiting, T is 1.29304K
Waiting, T is 0.97316K
Temperature is 0.97316K
Waiting, T is 0.871051K
Waiting, T is 0.824819K
Waiting, T is 0.843555K
Waiting, T is 0.802878K
Waiting, T is 0.779562K
Waiting, T is 0.764063K
Waiting, T is 0.761131K
Waiting, T is 0.774225K
Waiting, T is 0.756979K
Waiting, T is 0.745802K

# Power sweep

In [7]:
for k in range(1,4*3600):
    sleep(1)
VNA.receiver_gain('Low')
power_path = r"\Power dependence"
filename =  "PS"
data_pth = base_path + power_path


tempo = 0.0
#tempo = lake.ch06.temperature()
VNA.powersweep(0,-75,26, central_value, span_value, 'cs', 2.5e3 + 1, 5e2, data_pth, filename, tempo)


Right now, we are measuring spectrum at 0.0dBm
Right now, we are measuring spectrum at -3.0dBm
Right now, we are measuring spectrum at -6.0dBm
Right now, we are measuring spectrum at -9.0dBm
Right now, we are measuring spectrum at -12.0dBm
Right now, we are measuring spectrum at -15.0dBm
Right now, we are measuring spectrum at -18.0dBm
Right now, we are measuring spectrum at -21.0dBm
Right now, we are measuring spectrum at -24.0dBm
Right now, we are measuring spectrum at -27.0dBm
Right now, we are measuring spectrum at -30.0dBm
Right now, we are measuring spectrum at -33.0dBm
Right now, we are measuring spectrum at -36.0dBm
Right now, we are measuring spectrum at -39.0dBm
Right now, we are measuring spectrum at -42.0dBm
Right now, we are measuring spectrum at -45.0dBm
Right now, we are measuring spectrum at -48.0dBm
Right now, we are measuring spectrum at -51.0dBm
Right now, we are measuring spectrum at -54.0dBm
Right now, we are measuring spectrum at -57.0dBm
Right now, we are measuri

# Temperature sweep

## PID sweep and characterisation

In [16]:
values_low = np.arange(10,100,1)
values_mid = np.arange(100,200,2.5)
values_high = np.arange(201, 500, 5)
values_huge = np.arange(500, 710, 10)
values = np.concatenate((values_low, values_mid, values_high, values_huge))
values


array([ 10. ,  11. ,  12. ,  13. ,  14. ,  15. ,  16. ,  17. ,  18. ,
        19. ,  20. ,  21. ,  22. ,  23. ,  24. ,  25. ,  26. ,  27. ,
        28. ,  29. ,  30. ,  31. ,  32. ,  33. ,  34. ,  35. ,  36. ,
        37. ,  38. ,  39. ,  40. ,  41. ,  42. ,  43. ,  44. ,  45. ,
        46. ,  47. ,  48. ,  49. ,  50. ,  51. ,  52. ,  53. ,  54. ,
        55. ,  56. ,  57. ,  58. ,  59. ,  60. ,  61. ,  62. ,  63. ,
        64. ,  65. ,  66. ,  67. ,  68. ,  69. ,  70. ,  71. ,  72. ,
        73. ,  74. ,  75. ,  76. ,  77. ,  78. ,  79. ,  80. ,  81. ,
        82. ,  83. ,  84. ,  85. ,  86. ,  87. ,  88. ,  89. ,  90. ,
        91. ,  92. ,  93. ,  94. ,  95. ,  96. ,  97. ,  98. ,  99. ,
       100. , 102.5, 105. , 107.5, 110. , 112.5, 115. , 117.5, 120. ,
       122.5, 125. , 127.5, 130. , 132.5, 135. , 137.5, 140. , 142.5,
       145. , 147.5, 150. , 152.5, 155. , 157.5, 160. , 162.5, 165. ,
       167.5, 170. , 172.5, 175. , 177.5, 180. , 182.5, 185. , 187.5,
       190. , 192.5,

In [9]:

temperature_path = r"\Temperature dependence\Low_T"
filename =  "TS"
data_pth = base_path + temperature_path

##### PID heating
P = 325
I = 10
D = 0

heater.P(P)
heater.I(I)
heater.D(D)
heater.input_channel(6)      # measure and read from MXC temperature sensor
sleep(1)

print("P = ", heater.P())
print("I = ", heater.I())
print("D = ", heater.D())

VNA.receiver_gain('Low')

heater.range_limits((np.array([ 7       , 8   , 10    , 30  , 90     , 300   , 901    , 1200 ])/1000).tolist() ) # units of K
#              corresponding to  31.6uA , 100uA , 316uA , 1mA , 3.16mA , 10mA  , 31.6mA , 100mA on a 120Ohm resistor, empirically determined


for _setpoint in values: # units of mK
    if __setpoint in value_med:
        # do stuff
    elif _setpoint in value_high:
        # wait something else
    elif _setpoitn in value_huge:
        # probably wait a bit longer
                
    print(f"Setpoint is set to {_setpoint} mK")
    heater.setpoint(_setpoint*1e-3)
    
    old_range_value = heater.output_range()  
    heater.set_range_from_temperature_adj(_setpoint*1e-3, lake.ch06.temperature(), heating_time = 5)
    new_range_value = heater.output_range() 
    
    if old_range_value != new_range_value:
        print(f"Setting output range to: {heater.output_range()}")
    
    heater.wait_tolerance(1e-2)       # doesn't do anything except in conjunction with other functions
                                      # tolerance (val: 0 - 1):if abs(t_reading - t_setpoint) / t_reading > tolerance: do stuff
    heater.wait_equilibration_time(6) # doesn't do anything except in conjunction with other functions
                                      # Duration during which temperature has to be within tolerance
    heater.wait_cycle_time(0.1)       # doesn't do anything except in conjunction with other functions
                                      # this time is being waited between the readings
    heater.mode('closed_loop')
                # Open loop: directly set output using Manual output and range parameters -> constant current to load
                # closed loop: The Closed Loop PID mode is the most commonly used closed loop control mode for  tightly controlling 
                # temperature using the heater outputs of the Model 336. In this mode the controller attempts to keep the load 
                # at exactly the user-entered setpoint temperature. To do this, it uses feedback from the control input sensor to calculate
                # and actively adjust the output.
    heater.wait_until_set_point_reached()

    # We reached the setpoint!
    print(f'MXC is at temperature of {lake.ch06.temperature()} {lake.ch06.units()}')
   

    # read in the data
    temperatureMXC_1 = lake.ch06.temperature()
    print(f'Measuring of MXC: {temperatureMXC_1} {lake.ch06.units()}\n')

    VNA.measure_S21(central_value, span_value, 2.5e3, -10, 5, 1e4, 'cs')
    temperatureMXC_2 = lake.ch06.temperature()
    VNA.save_data(data_pth, filename, round(1e3*np.average([temperatureMXC_1,temperatureMXC_2]),2))
    
    
    
    t_time           = time.time()
    temperatureMXC   = round(1e3*np.average([temperatureMXC_1,temperatureMXC_2]),2)
    T_sweep_data     = np.column_stack((t_time,temperatureMXC))
    with open(data_pth + '_' + filename + '.txt', 'a+') as f:
        np.savetxt(f, T_sweep_data)
        f.flush()


heater.output('off')
heater.mode('off')
print("done!")

P =  325.0
I =  10.0
D =  0.0
Setpoint is set to 10.0
MXC is at temperature of 0.010021 kelvin
Measuring of MXC: 0.010021 kelvin

Setpoint is set to 11.0
Setting output range to: 1mA
MXC is at temperature of 0.0110084 kelvin
Measuring of MXC: 0.0110084 kelvin

Setpoint is set to 12.0
MXC is at temperature of 0.0120017 kelvin
Measuring of MXC: 0.0120017 kelvin

Setpoint is set to 13.0
MXC is at temperature of 0.0130124 kelvin
Measuring of MXC: 0.0130124 kelvin

Setpoint is set to 14.0
MXC is at temperature of 0.0139888 kelvin
Measuring of MXC: 0.0139861 kelvin

Setpoint is set to 15.0
MXC is at temperature of 0.014993 kelvin
Measuring of MXC: 0.014993 kelvin

Setpoint is set to 16.0
MXC is at temperature of 0.0159853 kelvin
Measuring of MXC: 0.0159853 kelvin

Setpoint is set to 17.0
MXC is at temperature of 0.0169634 kelvin
Measuring of MXC: 0.01696 kelvin

Setpoint is set to 18.0
MXC is at temperature of 0.017953 kelvin
Measuring of MXC: 0.017953 kelvin

Setpoint is set to 19.0
MXC is 

In [ ]:
heater.output('off')
heater.mode('off')

## Basic forced temperature sweep

In [10]:
VNA.receiver_gain('Low')

temperature_path = r"\Temperature dependence\Low_T"
filename =  "TS"
data_pth = base_path + temperature_path


heater.mode('open_loop')

for pwr in ['3.16mA','10mA']:
    i = 6
    sleep(2)
    heater.output_range(pwr)
    sleep(2)
    
    print("set range:" + heater.output_range() + '\n')
    while i < 50:
        heater.output(i)
        sleep(1)
        k = 0
        # wait 3 minutes for equillibrium
        print("Output is {}%".format(heater.output()))
        for k in range(0, 15):
            sleep(10)
            print(lake.ch06.temperature())
            
        # measure things, take average of temperatures before and after
        temperatureMXC_1 = lake.ch06.temperature()
        VNA.measure_S21(central_value, span_value, 2.5e3, -10, 10, 1e4, 'cs')
        temperatureMXC_2 = lake.ch06.temperature()
        VNA.save_data(data_pth, filename, round(1e3*np.average([temperatureMXC_1,temperatureMXC_2]),2))
        
        t_time           = time.time()
        temperatureMXC   = round(1e3*np.average([temperatureMXC_1,temperatureMXC_2]),2)
        T_sweep_data = np.column_stack((t_time,temperatureMXC))
        with open(data_pth + filename + '.txt', 'a+') as f:
            np.savetxt(f, T_sweep_data)
            f.flush()
        i += 2
        sleep(1)
        heater.output(i)

sleep(3)
i = 11
sleep(3)
heater.output_range('31.6mA')
sleep(3)

print("set range:" + heater.output_range() + '\n')
while i < 26:
    heater.output(i)
    k = 0
    # wait 4 minutes for equillibrium
    # a bit more dangerous at high heat so show temp more often
    print("Output is {}%".format(heater.output()))
    for k in range(0, 80):
        sleep(5.5)
        print(lake.ch06.temperature())
    
    # measure things, take average of temperatures before and after
    temperatureMXC_1 = lake.ch06.temperature()
    VNA.measure_S21(central_value, span_value, 2.5e3, -10, 10, 1e4, 'cs')
    temperatureMXC_2 = lake.ch06.temperature()
    VNA.save_data(data_pth, filename, round(1e3*np.average( [temperatureMXC_1,temperatureMXC_2] ), 2))
    
    t_time           = time.time()
    temperatureMXC   = round(1e3*np.average([temperatureMXC_1,temperatureMXC_2]),2)
    T_sweep_data = np.column_stack((t_time,temperatureMXC))
    with open(data_pth + filename + '.txt', 'a+') as f:
        np.savetxt(f, T_sweep_data)
        f.flush()
    
    i += 0.4
    sleep(1)
    heater.output(i)
                  
heater.output('off')

set range:3.16mA

Output is 6.0%
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
Output is 7.0%
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
Output is 8.0%
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.007002
0.007226
0.0074415
0.0077278
0.0079971
Output is 9.0%
0.0084378
0.0086493
0.0088244
0.0090273
0.0091962
0.009378
0.0095613
0.0097363
0.0099173
0.0100835
0.0102457
0.0104043
0.0105584
0.0107169
0.0108387
Output is 10.0%
0.0111743
0.0113305
0.0114792
0.0116209
0.0117904
0.0119412
0.0121
0.0122547
0.0123779
0.0125209
0.0126541
0.0127928
0.0129163
0.0130471
0.0131397
Output is 11.0%
0.0134304
0.0135465
0.0136844
0.0138133
0.013951
0.0140797
0.0142172
0.0143454
0.0144775
0.0145886
0.0147094
0.0148167
0.014909
0.0150097
0.0151048
Output is 12.0%
0.0153199
0.0154342
0.0155662
0.0156846
0.0158054
0.0159258
0.0160393
0.0161597
0.0162785
0.0163771
0.0164753
0.0165744
0.016668
0.0167403
0.0168246
Output is 13.0%
0.0169949
0.01711
0.0172444
0.0173816
0.0175009
0.0175891

KeyboardInterrupt: 

In [12]:
for pwr in ['10mA']:
    i = 10
    sleep(2)
    heater.output_range(pwr)
    sleep(2)
    
    print("set range:" + heater.output_range() + '\n')
    while i < 50:
        heater.output(i)
        sleep(1)
        k = 0
        # wait 3 minutes for equillibrium
        print("Output is {}%".format(heater.output()))
        for k in range(0, 15):
            sleep(10)
            print(lake.ch06.temperature())
            
        # measure things, take average of temperatures before and after
        temperatureMXC_1 = lake.ch06.temperature()
        VNA.measure_S21(central_value, span_value, 2.5e3, -10, 10, 1e4, 'cs')
        temperatureMXC_2 = lake.ch06.temperature()
        VNA.save_data(data_pth, filename, round(1e3*np.average([temperatureMXC_1,temperatureMXC_2]),2))
        
        t_time           = time.time()
        temperatureMXC   = round(1e3*np.average([temperatureMXC_1,temperatureMXC_2]),2)
        T_sweep_data = np.column_stack((t_time,temperatureMXC))
        with open(data_pth + filename + '.txt', 'a+') as f:
            np.savetxt(f, T_sweep_data)
            f.flush()
        i += 2
        sleep(1)
        heater.output(i)

sleep(3)
i = 11
sleep(3)
heater.output_range('31.6mA')
sleep(3)

print("set range:" + heater.output_range() + '\n')
while i < 26:
    heater.output(i)
    k = 0
    # wait 4 minutes for equillibrium
    # a bit more dangerous at high heat so show temp more often
    print("Output is {}%".format(heater.output()))
    for k in range(0, 80):
        sleep(5.5)
        print(lake.ch06.temperature())
    
    # measure things, take average of temperatures before and after
    temperatureMXC_1 = lake.ch06.temperature()
    VNA.measure_S21(central_value, span_value, 2.5e3, -10, 10, 1e4, 'cs')
    temperatureMXC_2 = lake.ch06.temperature()
    VNA.save_data(data_pth, filename, round(1e3*np.average( [temperatureMXC_1,temperatureMXC_2] ), 2))
    
    t_time           = time.time()
    temperatureMXC   = round(1e3*np.average([temperatureMXC_1,temperatureMXC_2]),2)
    T_sweep_data = np.column_stack((t_time,temperatureMXC))
    with open(data_pth + filename + '.txt', 'a+') as f:
        np.savetxt(f, T_sweep_data)
        f.flush()
    
    i += 0.4
    sleep(1)
    heater.output(i)
                  
heater.output('off')

set range:10mA

Output is 10.0%
0.0436563
0.0438494
0.0439223
0.0439794
0.0440031
0.0440263
0.0440399
0.0440559
0.0440498
0.044082
0.0440866
0.0440862
0.0441239
0.044099
0.0441132
Output is 12.0%
0.0453066
0.0459095
0.0463277
0.0467388
0.0470761
0.0474112
0.0477103
0.0479968
0.048266
0.0485007
0.0487427
0.0489477
0.0491822
0.0493494
0.0495379
Output is 14.0%
0.0512132
0.0519834
0.0525146
0.0530921
0.0535347
0.0539726
0.0543721
0.0547344
0.0550816
0.0554255
0.0557757
0.0561
0.0563899
0.0566335
0.056922
Output is 16.0%
0.0587018
0.0594837
0.060108
0.0607347
0.0612676
0.0617504
0.0622368
0.0626487
0.0630978
0.06344
0.0638188
0.0641489
0.0644357
0.0647561
0.0650398
Output is 18.0%
0.0670376
0.0678632
0.0684791
0.0690827
0.0696073
0.0701561
0.0706219
0.0711144
0.0715541
0.07195
0.0723668
0.0727321
0.073073
0.0733543
0.0736763
Output is 20.0%
0.0756783
0.0765596
0.0772868
0.0778656
0.078431
0.0789805
0.0794532
0.0799006
0.0803806
0.0808188
0.0813002
0.0816649
0.0820142
0.0823169
0.082723
Out

In [15]:
heater.output_range('off')
heater.output('off')
heater.mode('off')